In [2]:
import pandas as pd
smiles_list = pd.read_csv('data/pic50_greater_equal_9.0.csv')['smiles'][:50]

In [4]:
smiles_list

0     CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nc(N4CCOCC4)cc...
1     Nc1ncc(-c2cccc(N3CCOCC3)c2)nc1C(=O)Nc1ncccc1N1...
2     Nc1ncc(-c2cccc3[nH]ncc23)nc1C(=O)Nc1ncccc1N1CC...
3     Cc1cc2c(-c3cnc(N)c(C(=O)Nc4ncccc4N4CCC(N)CC4)n...
4     CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nc(N4CCC(F)(F)...
5     CN(C)c1ccc(C(F)(F)F)c(-c2cnc(N)c(C(=O)Nc3ncccc...
6     Nc1ncc(-c2cccc3[nH]c(C(F)(F)F)cc23)nc1C(=O)Nc1...
7     Nc1ncc(-c2nc(N3CCOCC3)ccc2C(F)(F)F)nc1C(=O)Nc1...
8     Nc1ncc(-c2nc(N3CCOCC3)cc3ccccc23)nc1C(=O)Nc1nc...
9     Nc1ncc(-c2cc(N3CCOCC3)cc(C(F)(F)F)c2)nc1C(=O)N...
10        Nc1ncc(-c2ccccc2)nc1C(=O)Nc1cnccc1N1CCC(N)CC1
11       Nc1ncc(-c2ccccc2F)nc1C(=O)Nc1cnccc1N1CCC(N)CC1
12    CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nc(C4=CCOCC4)n...
13    Nc1ncc(-c2cc(N3CCOCC3)ccc2C(F)(F)F)nc1C(=O)Nc1...
14    Nc1ncc(-c2cccc3[nH]ccc23)nc1C(=O)Nc1ncccc1N1CC...
15        Nc1ncc(-c2ccccc2)nc1C(=O)Nc1ccccc1N1CCC(N)CC1
16    CCOCC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3ncccc3OC(F)...
17    CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3cc(N4CCOCC

In [5]:
import warnings
from pathlib import Path
import subprocess
from itertools import product
import numpy as np
import pandas as pd  # 用于记录 CSV 结果
from MDAnalysis import Universe
from openbabel import pybel
import os

# Filter warnings
warnings.filterwarnings("ignore")

# 设置工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data'
DATA.mkdir(parents=True, exist_ok=True)

# CSV 文件路径
RESULT_CSV = DATA / 'result_dock.csv'

# 初始化 CSV 文件
if not RESULT_CSV.exists():
    pd.DataFrame(columns=['pdb', 'ligand_resnames', 'smiles', 'dock_info']).to_csv(RESULT_CSV, index=False)


class Structure(Universe):
    """Core object to load structures with."""

    @classmethod
    def from_string(cls, pdb_path):
        """Load a structure from a local PDB file."""
        return cls(pdb_path)


# 提取蛋白质并保存为 PDB 格式
def extract_protein_to_pdb(structure, protein_path):
    """Extract protein from structure and save it as PDB."""
    protein = structure.select_atoms("protein")
    protein.write(str(protein_path))  # 自动根据扩展名保存为 PDB 格式


# 自动检测所有配体残基
def find_all_ligand_resnames(structure):
    """返回所有非蛋白质和非水分子的残基名称列表"""
    ligand_atoms = structure.select_atoms("not protein and not resname HOH")
    return list(set(ligand_atoms.resnames))


def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """Convert a PDB file to a PDBQT file."""
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)


def smiles_to_pdbqt(smiles, pdbqt_path, pH=7.4):
    """Convert a SMILES string to a PDBQT file."""
    molecule = pybel.readstring("smi", smiles)
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    molecule.make3D(forcefield="mmff94s", steps=10000)
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)


def run_smina(ligand_path, protein_path, out_path, pocket_center, pocket_size):
    """Perform docking with Smina."""
    output_text = subprocess.check_output([
        "smina",
        "--receptor", str(protein_path),
        "--ligand", str(ligand_path),
        "--out", str(out_path),
        "--center_x", str(pocket_center[0]),
        "--center_y", str(pocket_center[1]),
        "--center_z", str(pocket_center[2]),
        "--size_x", str(pocket_size[0]),
        "--size_y", str(pocket_size[1]),
        "--size_z", str(pocket_size[2])
    ])
    return output_text.decode("utf-8")


def parse_dock_info(dock_output):
    """解析 Smina 输出中的对接信息"""
    dock_lines = dock_output.splitlines()
    dock_data = []
    capture = False
    for line in dock_lines:
        if "mode" in line and "affinity" in line:  # 开始捕获表格数据
            capture = True
            continue
        if capture:
            if line.strip() == "" or "Refine time" in line or "Loop time" in line:
                break
            dock_data.append(line.strip())
    return "\n".join(dock_data)


def split_sdf_file(sdf_path):
    """Split an SDF file into separate files for each molecule."""
    sdf_path = Path(sdf_path)
    stem = sdf_path.stem
    parent = sdf_path.parent
    molecules = pybel.readfile("sdf", str(sdf_path))
    for i, molecule in enumerate(molecules, 1):
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)


# 主程序
pdb_list = ['7N8T', '8ELC', '4RA4', '8U37', '8UAK']
# 读取data里面的pic50_greater_equal_9.0.csv文件，获取其中的smiles，前面的50个
smiles_list = pd.read_csv('data/pic50_greater_equal_9.0.csv')['smiles'][:50]
# smiles_list = [
#     "CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nc(N4CCOCC4)cc4ccccc34)cnc2N)CC1",
#     "Nc1ncc(-c2cccc(N3CCOCC3)c2)nc1C(=O)Nc1ncccc1N1CCC(N)CC1",
# ]

# 遍历所有 PDB 和 SMILES 的笛卡尔积
for pdb_id, smiles in product(pdb_list, smiles_list):
    # 创建每个 PDB 的独立目录
    pdb_dir = DATA / f"data_{pdb_id}"
    pdb_dir.mkdir(parents=True, exist_ok=True)

    # 定义 PDB 文件路径
    pdb_path = pdb_dir / f"{pdb_id}.pdb"

    # 加载结构
    structure = Structure.from_string(pdb_path)

    # 提取并保存蛋白质部分
    protein_path = pdb_dir / "protein.pdb"
    extract_protein_to_pdb(structure, protein_path)

    # 转换蛋白质为 PDBQT 格式
    protein_pdbqt_path = pdb_dir / "protein.pdbqt"
    pdb_to_pdbqt(protein_path, protein_pdbqt_path)

    # 检测所有 ligand
    all_ligands = find_all_ligand_resnames(structure)
    print(f"{pdb_id} - Detected Ligands: {all_ligands}")

    # 对所有 ligand 进行处理
    for ligand_resname in all_ligands:
        # 创建 ligand-specific 子目录
        ligand_dir = pdb_dir / f"ligand_{ligand_resname}"
        ligand_dir.mkdir(parents=True, exist_ok=True)

        # 自动检测口袋中心和大小
        ligand = structure.select_atoms(f"resname {ligand_resname}")
        print(f"Processing {pdb_id} {smiles} {ligand_resname}")
        pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2
        pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 5

        # 创建针对 SMILES 的子目录
        smiles_hash = smiles.replace("(", "").replace(")", "").replace("=", "").replace("-", "").replace("/", "").replace("\\", "").replace(".", "").replace(",", "").replace(":", "")
        smiles_dir = ligand_dir / f"smiles_{smiles_hash}"
        smiles_dir.mkdir(parents=True, exist_ok=True)

        # 定义文件路径
        ligand_path = smiles_dir / "ligand.pdbqt"
        docking_out_path = smiles_dir / "ligand_docking.sdf"
        log_path = smiles_dir / "docking_log.txt"

        # 转换配体为 PDBQT 格式
        smiles_to_pdbqt(smiles, ligand_path)

        # 运行对接并保存日志
        docking_output = run_smina(ligand_path, protein_pdbqt_path, docking_out_path, pocket_center, pocket_size)
        with open(log_path, "w") as log_file:
            log_file.write(f"PDB: {pdb_id}\nSMILES: {smiles}\nLigand Resname: {ligand_resname}\n")
            log_file.write(f"Pocket Center: {pocket_center}\nPocket Size: {pocket_size}\n\nDocking Output:\n")
            log_file.write(docking_output)

        # 拆分 SDF 文件
        split_sdf_file(docking_out_path)

        # 提取对接结果并更新到 CSV
        dock_info = parse_dock_info(docking_output)
        new_row = {'pdb': pdb_id, 'ligand_resnames': ligand_resname, 'smiles': smiles, 'dock_info': dock_info}
        result_df = pd.read_csv(RESULT_CSV)
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)
        # 保存更新的结果到 CSV 文件
        result_df.to_csv(RESULT_CSV, index=False)

print("Docking workflow completed successfully!")


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/小辣椒/07-AIDD/分子对接/data/data_7N8T/protein.pdb)



7N8T - Detected Ligands: ['AMP', 'P6G']
Processing 7N8T CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nc(N4CCOCC4)cc4ccccc34)cnc2N)CC1 AMP
Processing 7N8T CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nc(N4CCOCC4)cc4ccccc34)cnc2N)CC1 P6G


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/小辣椒/07-AIDD/分子对接/data/data_7N8T/protein.pdb)



7N8T - Detected Ligands: ['AMP', 'P6G']
Processing 7N8T Nc1ncc(-c2cccc(N3CCOCC3)c2)nc1C(=O)Nc1ncccc1N1CCC(N)CC1 AMP
Processing 7N8T Nc1ncc(-c2cccc(N3CCOCC3)c2)nc1C(=O)Nc1ncccc1N1CCC(N)CC1 P6G


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/小辣椒/07-AIDD/分子对接/data/data_7N8T/protein.pdb)



...

8UAK - Detected Ligands: ['W39', 'TPO']
Processing 8UAK NC(=O)c1ccccc1Nc1nc(Nc2ccc(N3CCOCC3)cc2F)ncc1Cl W39
Processing 8UAK NC(=O)c1ccccc1Nc1nc(Nc2ccc(N3CCOCC3)cc2F)ncc1Cl TPO


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/小辣椒/07-AIDD/分子对接/data/data_8UAK/protein.pdb)



8UAK - Detected Ligands: ['W39', 'TPO']
Processing 8UAK CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nccc(OC(F)(F)F)c3C#N)cnc2N)CC1 W39
Processing 8UAK CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nccc(OC(F)(F)F)c3C#N)cnc2N)CC1 TPO


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/小辣椒/07-AIDD/分子对接/data/data_8UAK/protein.pdb)



8UAK - Detected Ligands: ['W39', 'TPO']
Processing 8UAK CCC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3ncccc3OC(F)(F)F)cnc2N)CC1 W39
Processing 8UAK CCC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3ncccc3OC(F)(F)F)cnc2N)CC1 TPO
Docking workflow completed successfully!


FileNotFoundError: [Errno 2] No such file or directory: '7N8T.pdb'

In [ ]:
import warnings
from pathlib import Path
import subprocess
from itertools import product
import numpy as np
import pandas as pd  # 用于记录 CSV 结果
from MDAnalysis import Universe
from openbabel import pybel
import os

# Filter warnings
warnings.filterwarnings("ignore")

# 设置工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data'
DATA.mkdir(parents=True, exist_ok=True)

# CSV 文件路径
RESULT_CSV = DATA / 'result_dock.csv'

# 初始化 CSV 文件
if not RESULT_CSV.exists():
    pd.DataFrame(columns=['pdb', 'ligand_resnames', 'smiles', 'dock_info']).to_csv(RESULT_CSV, index=False)


class Structure(Universe):
    """Core object to load structures with."""

    @classmethod
    def from_string(cls, pdb_path):
        """Load a structure from a local PDB file."""
        return cls(pdb_path)


# 提取蛋白质并保存为 PDB 格式
def extract_protein_to_pdb(structure, protein_path):
    """Extract protein from structure and save it as PDB."""
    protein = structure.select_atoms("protein")
    protein.write(str(protein_path))  # 自动根据扩展名保存为 PDB 格式


# 自动检测所有配体残基
def find_all_ligand_resnames(structure):
    """返回所有非蛋白质和非水分子的残基名称列表"""
    ligand_atoms = structure.select_atoms("not protein and not resname HOH")
    return list(set(ligand_atoms.resnames))


def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """Convert a PDB file to a PDBQT file."""
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)


def smiles_to_pdbqt(smiles, pdbqt_path, pH=7.4):
    """Convert a SMILES string to a PDBQT file."""
    molecule = pybel.readstring("smi", smiles)
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    molecule.make3D(forcefield="mmff94s", steps=10000)
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)


def run_smina(ligand_path, protein_path, out_path, pocket_center, pocket_size):
    """Perform docking with Smina."""
    output_text = subprocess.check_output([
        "smina",
        "--receptor", str(protein_path),
        "--ligand", str(ligand_path),
        "--out", str(out_path),
        "--center_x", str(pocket_center[0]),
        "--center_y", str(pocket_center[1]),
        "--center_z", str(pocket_center[2]),
        "--size_x", str(pocket_size[0]),
        "--size_y", str(pocket_size[1]),
        "--size_z", str(pocket_size[2])
    ])
    return output_text.decode("utf-8")


def parse_dock_info(dock_output):
    """解析 Smina 输出中的对接信息"""
    dock_lines = dock_output.splitlines()
    dock_data = []
    capture = False
    for line in dock_lines:
        if "mode" in line and "affinity" in line:  # 开始捕获表格数据
            capture = True
            continue
        if capture:
            if line.strip() == "" or "Refine time" in line or "Loop time" in line:
                break
            dock_data.append(line.strip())
    return "\n".join(dock_data)


def split_sdf_file(sdf_path):
    """Split an SDF file into separate files for each molecule."""
    sdf_path = Path(sdf_path)
    stem = sdf_path.stem
    parent = sdf_path.parent
    molecules = pybel.readfile("sdf", str(sdf_path))
    for i, molecule in enumerate(molecules, 1):
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)
        
        
def run_smina_with_output(ligand_path, protein_path, out_path, pocket_center, pocket_size):
    """Perform docking with Smina and include the output in a combined PDBQT format."""
    output_log = subprocess.check_output([
        "smina",
        "--receptor", str(protein_path),
        "--ligand", str(ligand_path),
        "--out", str(out_path),
        "--center_x", str(pocket_center[0]),
        "--center_y", str(pocket_center[1]),
        "--center_z", str(pocket_center[2]),
        "--size_x", str(pocket_size[0]),
        "--size_y", str(pocket_size[1]),
        "--size_z", str(pocket_size[2])
    ])
    return output_log.decode("utf-8")

# 主程序
pdb_list = ['7N8T', '8ELC', '4RA4', '8U37', '8UAK']
smiles_list = [
    "CC1(N)CCN(c2cccnc2NC(=O)c2nc(-c3nc(N4CCOCC4)cc4ccccc34)cnc2N)CC1",
    "Nc1ncc(-c2cccc(N3CCOCC3)c2)nc1C(=O)Nc1ncccc1N1CCC(N)CC1",
]

# Update the docking loop to ensure output alignment with your requirements
for pdb_id, smiles in product(pdb_list, smiles_list):
    pdb_dir = DATA / f"data_{pdb_id}"
    pdb_dir.mkdir(parents=True, exist_ok=True)

    pdb_path = pdb_dir / f"{pdb_id}.pdb"
    structure = Structure.from_string(pdb_path)

    protein_path = pdb_dir / "protein.pdb"
    extract_protein_to_pdb(structure, protein_path)

    protein_pdbqt_path = pdb_dir / "protein.pdbqt"
    pdb_to_pdbqt(protein_path, protein_pdbqt_path)

    for ligand_resname in find_all_ligand_resnames(structure):
        ligand_dir = pdb_dir / f"ligand_{ligand_resname}"
        ligand_dir.mkdir(parents=True, exist_ok=True)

        ligand = structure.select_atoms(f"resname {ligand_resname}")
        pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2
        pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 5

        smiles_hash = ''.join(e for e in smiles if e.isalnum())
        smiles_dir = ligand_dir / f"smiles_{smiles_hash}"
        smiles_dir.mkdir(parents=True, exist_ok=True)

        ligand_path = smiles_dir / "ligand.pdbqt"
        docking_out_path = smiles_dir / f"{pdb_id}_{smiles_hash}_dock.pdbqt"
        log_path = smiles_dir / "docking_log.txt"

        smiles_to_pdbqt(smiles, ligand_path)

        docking_output = run_smina_with_output(ligand_path, protein_pdbqt_path, docking_out_path, pocket_center, pocket_size)
        
        with open(log_path, "w") as log_file:
            log_file.write(docking_output)

        dock_info = parse_dock_info(docking_output)
        new_row = {
            'pdb': pdb_id,
            'ligand_resnames': ligand_resname,
            'smiles': smiles,
            'dock_info': dock_info
        }
        result_df = pd.read_csv(RESULT_CSV)
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)
        result_df.to_csv(RESULT_CSV, index=False)

print("Docking workflow completed successfully!")


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/小辣椒/07-AIDD/分子对接/data/data_7N8T/protein.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/小辣椒/07-AIDD/分子对接/data/data_7N8T/protein.pdb)

